In [1]:
%load_ext autoreload
%autoreload 2

import xarray as xr
import rydanalysis as ra
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path
import numpy as np
from tqdm.notebook import tqdm#
from scipy.stats import sem
from scipy.integrate import quad
from lmfit import Parameter

%matplotlib widget

In [4]:
path = Path.cwd().parent / "raw_data"

live_analysis_path = Path.cwd().parent / "Analysis" 
peaks_file = live_analysis_path / "peaks.h5"

In [5]:
data, peak_df = ra.update_data(path, csv_path="peak_df_2.csv")

2021-05-24 22:44:31.136 INFO    numexpr.utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-05-24 22:44:31.137 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


Found exisiting peak_df.


## Analysis

In [24]:
variable = data.ryd_data.variables.columns[0]

summary = ra.summarize_peak_description(peak_df)
counts = summary.counts.to_series()
counts_mean = counts.groupby(variable).mean()
counts_sem = counts.groupby(variable).sem(ddof=1)

counts_summary = pd.concat([counts_mean, counts_sem], axis=1)
counts_summary.columns = ["counts", "counts_sem"]
counts_summary = counts_summary.dropna()

In [7]:
fig, ax = plt.subplots()

ax.errorbar(counts_mean.index, counts_mean, counts_sem, linestyle='-', marker='o')
ax.set_xlabel(variable)
ax.set_ylabel("ion counts")

fig.savefig("raw_data.png")
#ax.set_xscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
model = ra.DampedCosineModel()
f, fft = model.get_fft(counts_mean.values, x=counts_mean.index)
fft[0] = 0
params = model.guess(counts_mean.values, x=counts_mean.index)

fig, ax = plt.subplots()
ax.plot(f, np.abs(fft)**2)
ax.errorbar([params["freq"].value], [params["amp"].value**2], xerr=[params["gamma"].value], linestyle='', marker='o')
ax.set_xlabel("frequency [µs]")
ax.set_ylabel("power spectrum [MHz^2]")
fig.savefig("power_spectrum.png", dpi=600, bbox_inches="tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
cosine_model = ra.DampedCosineModel()
cosine_params = cosine_model.guess(counts_summary.counts.values, x=counts_summary.index)
cosine_params["amp"].min = 0
cosine_params["phase"] = Parameter("phase", value=180, vary=False)

fitres = cosine_model.fit(
    counts_summary.counts,
    params=cosine_params,
    x=counts_mean.index,
    weights=1/counts_summary.counts_sem
)

fig, ax = plt.subplots()
fitres.plot_fit(ax=ax, numpoints=100, show_init=False)
times = np.linspace(0, 0.2, 110)
plt.text(
    0.15, 0.95,
    'Omega: %.5g  $\pm$ %.5g gamma: %.5g  $\pm$ %.5g'%(fitres.params['freq'].value, fitres.params['freq'].stderr, fitres.params['gamma'].value, fitres.params['gamma'].stderr),
    transform=ax.transAxes)
ax.set_xlabel(variable + " [us]")
ax.set_ylabel("ion count")

plt.savefig('48P_%s_Fit_Cosine.png'%variable, dpi=600) # add leading zero to the figure name

print("chi^2 reduced", fitres.redchi)
fitres

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

chi^2 reduced 4.8784924092325035


In [45]:
1/(4 * fitres.params["freq"])

0.05920842371327371